In [1]:
!pip install langchain langchain-community langchain-core

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/bitsandbytes-0.45.4.dev0-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/looseversion-1.3.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_utilities-0.12.0.dev0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading eg

In [1]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import os
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
import pandas as pd
import pytorch_lightning as pl
import torch
torch.set_float32_matmul_precision('medium')
import torch.nn as nn
import glob
from transformers import ElectraModel, AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split

In [2]:
###########################
# KOTE 파인튜닝 모델 로드 (체크포인트 파일 경로 수정)
###########################
best_ckpt_path_kote = './model/250127_KcElectra_kote.ckpt' # Colab 경로에 맞게 수정
print("Best checkpoint path (KOTE Finetuned):", best_ckpt_path_kote)

Best checkpoint path (KOTE Finetuned): ./model/250127_KcElectra_kote.ckpt


In [3]:
# KOTETagger 클래스는 이전 코드와 동일
class KOTETagger(pl.LightningModule): # KOTETagger 클래스 정의 (이전 코드에서 복사)
    def __init__(self, n_training_steps=None, n_warmup_steps=None):
        super().__init__()
        self.electra = AutoModel.from_pretrained(MODEL_NAME, return_dict=True) # pretrained_electra 제거 및 직접 로드
        self.classifier = nn.Linear(self.electra.config.hidden_size, 44) # num_labels=44 (KOTE 라벨 개수)
        self.n_training_steps = n_training_steps
        self.n_warmup_steps = n_warmup_steps
        self.criterion = nn.BCELoss()

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.electra(input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]
        logits = self.classifier(cls_output)
        probs = torch.sigmoid(logits)
        loss = 0
        if labels is not None:
            loss = self.criterion(probs, labels)
        return loss, probs

    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, _ = self(input_ids, attention_mask, labels)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        return {"val_loss": loss}

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=INITIAL_LR, weight_decay=WEIGHT_DECAY) # 가중치 감쇠
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=self.n_warmup_steps,
            num_training_steps=self.n_training_steps
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val_loss",
                "interval": "step",
                "frequency": 1
            }
        }

In [4]:
###########################
# 토크나이저 로드
###########################
MODEL_NAME = "beomi/KcELECTRA-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [5]:
emotion_labels = [
    "감동/감탄", "경악", "고마움", "공포/무서움", "귀찮음", "기대감", "기쁨", "깨달음",
    "놀람", "당황/난처", "부끄러움", "부담/안_내킴", "불쌍함/연민", "불안/걱정", "불평/불만",
    "비장함", "뿌듯함", "서러움", "슬픔", "신기함/관심", "아껴주는", "안심/신뢰", "안타까움/실망",
    "어이없음", "없음", "역겨움/징그러움", "우쭐댐/무시함", "의심/불신", "재미없음", "절망",
    "존경", "죄책감", "즐거움/신남", "증오/혐오", "지긋지긋", "짜증", "패배/자기혐오",
    "편안/쾌적", "한심함", "행복", "화남/분노", "환영/호의", "흐뭇함(귀여움/예쁨)", "힘듦/지침"
]

In [6]:
kote_finetuned_model = KOTETagger.load_from_checkpoint(best_ckpt_path_kote)
pretrained_electra = kote_finetuned_model.electra # 수정: electra backbone만 가져옴

In [7]:
###########################
# LightningModule 정의 (PoetryTagger) (기존 코드 활용 + 가중치 손실 함수, Dropout, Weight Decay, Learning Rate 감소, EarlyStopping patience 증가)
###########################
INITIAL_LR = 1e-5 # 학습률 감소 (원래 2e-5, 1e-5, 5e-6, 2e-6)
DROPOUT_RATE = 0.5 # Dropout 비율 (0.1, 0.3, 0.5) - Dropout 추가
WEIGHT_DECAY = 0.02 # Weight Decay 값 (0.001, 0.01, 0.02) - Weight Decay 추가
THRESHOLD = 0.3

class PoetryTagger(pl.LightningModule):
    def __init__(self, n_training_steps=None, n_warmup_steps=None, dropout_rate=DROPOUT_RATE): # dropout_rate hyperparameter
        super().__init__()
        self.electra = pretrained_electra # 수정: KOTE 파인튜닝 모델의 electra backbone 사용
        self.classifier = nn.Sequential( # nn.Sequential 사용하여 dropout layer 추가
            nn.Linear(self.electra.config.hidden_size, len(emotion_labels)),
            nn.Dropout(dropout_rate)
        ) # Classifier 출력층 크기 자동 조정
        self.n_training_steps = n_training_steps
        self.n_warmup_steps = n_warmup_steps
        self.criterion = nn.BCELoss() # 기본 BCE Loss (가중치 미적용)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.electra(input_ids, attention_mask=attention_mask)
        # [CLS] 토큰 기준으로 분류
        cls_output = outputs.last_hidden_state[:, 0, :]
        logits = self.classifier(cls_output)
        probs = torch.sigmoid(logits)

        loss = 0
        if labels is not None:
            loss = self.criterion(probs, labels)

        return loss, probs

    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]

        loss, _ = self(input_ids, attention_mask, labels) # forward 함수에 weights 제거

        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]

        loss, outputs = self(input_ids, attention_mask, labels) # validation loss는 기존 BCE Loss 사용 (optional)
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        return {"val_loss": loss} # validation metrics are optional

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=INITIAL_LR, weight_decay=WEIGHT_DECAY) # 가중치 감쇠
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=self.n_warmup_steps,
            num_training_steps=self.n_training_steps
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val_loss",
                "interval": "step",
                "frequency": 1
            }
        }

In [8]:
# ###########################
# # Best checkpoint load & Evaluation (poetry-finetuning) (기존 코드 활용)
# ###########################
def get_latest_version_dir_poetry(base_dir="./lightning_logs/poetry-finetuning-3agreements-only"): # poetry-weighted-finetuning 로 변경
    # version_0, version_1, ... version_50 경로를 모두 찾아 리스트업
    version_dirs = glob.glob(os.path.join(base_dir, "version_*"))
    # 버전 숫자를 기준으로 정렬
    version_dirs.sort(key=lambda x: int(x.split("_")[-1]))
    if not version_dirs:
        raise FileNotFoundError(f"No version_* directories found under '{base_dir}'")
    # 가장 마지막(숫자가 가장 큰) 버전 경로 반환
    return version_dirs[-1]

def get_latest_checkpoint_poetry(version_dir):
    ckpt_dir = os.path.join(version_dir, "checkpoints")
    ckpt_list = glob.glob(os.path.join(ckpt_dir, "*.ckpt"))
    ckpt_list.sort()  # 파일명 기준 정렬
    if not ckpt_list:
        raise FileNotFoundError(f"No .ckpt found under '{ckpt_dir}'")
    # 가장 마지막 파일(정렬 기준)
    return ckpt_list[-1]

In [9]:
# KPoEM 모델 로드 (local)
latest_version_dir_poetry = get_latest_version_dir_poetry("./lightning_logs/poetry-finetuning-3agreements-only") # poetry-weighted-finetuning 로 변경
best_ckpt_path_poetry = get_latest_checkpoint_poetry(latest_version_dir_poetry)
print("Best checkpoint path (Poetry Weighted Finetuned):", best_ckpt_path_poetry) # poetry-weighted-finetuning 로 변경

best_model_poetry = PoetryTagger.load_from_checkpoint(best_ckpt_path_poetry)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

best_model_poetry.to(device)
best_model_poetry.eval()
best_model_poetry.freeze()

Best checkpoint path (Poetry Weighted Finetuned): ./lightning_logs/poetry-finetuning-3agreements-only/version_0/checkpoints/epoch9-val_loss0.2566.ckpt


In [10]:
sample_text = """하루 종일 지친 몸으로만 떠돌다가
땅에 떨어져 죽지 못한
햇빛들은 줄지어 어디로 가는 걸까

웅성웅성 가장 근심스러운 색깔로 서행하며
이미 어둠이 깔리는 소각장으로 몰려들어
몇 점 폐휴지로 타들어가는 오루 6시의 참혹한 형량
단 한 번 후회도 용서하지 않는 무서운 시간
바람은 긴 채찍을 휘둘러
살아서 빛나는 온갖 상징을 몰아내고 있다.

도시는 곧 활자들이 일제히 빠져 달아나
속도 없이 페이지를 펄럭이는 텅 빈 한 권 책이 되리라.
승부를 알 수 없는 하루와의 싸움에서
우리는 패배했을까. 오늘도 물어보는 사소한 물음은
그러나 우리의 일생을 텅텅 흔드는 것.

오후 6시의 소각장 위로 말없이
검은 연기가 우산처럼 펼쳐지고
이젠 우리들의 차례였다.
두렵지 않은가.
밤이면 그림자를 빼앗겨 누구나 아득한 혼자였다.

문득 거리를 빠르게 스쳐가는 일상의 공포
보여다오. 지금까지 무엇을 했는가 살아 있는 그대여
오후 6시 우리들 이마에도 아, 붉은 노을이 떴다.

그러면 우리는 어디로 가지?
아직도 펄펄 살아 있는 우리는 이제 각자 어디로 가지?
""" # 기형도 - 노을

In [37]:
def classify_emotion(sample_text):

    encoding = tokenizer(
        sample_text,
        max_length=512,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )

    with torch.no_grad():
        # 입력 텐서 또한 같은 device로 이동
        input_ids = encoding["input_ids"].to(device)
        attention_mask = encoding["attention_mask"].to(device)

        # forward
        _, predictions = best_model_poetry(input_ids, attention_mask)  # best_model_poetry 사용

    # 추론 결과를 CPU로 가져와 numpy로 변환
    predictions = predictions.flatten().cpu().numpy()

    # 결과를 딕셔너리로 저장 (숫자값으로 변환)
    result_dict = {
        label_name: float(round(score, 3))  # np.float32 -> float 변환
        for label_name, score in zip(emotion_labels, predictions)
        if score > THRESHOLD
    }

    # 결과 출력
    # print("\n[Sample Inference 결과]")
    # print(result_dict)

    return result_dict
    # 예시 출력
    # {'불안/걱정': 0.336, '슬픔': 0.311}

In [38]:
classify_emotion(sample_text)


[Sample Inference 결과]
{'공포/무서움': 0.42800000309944153, '놀람': 0.30399999022483826, '당황/난처': 0.5070000290870667, '부담/안_내킴': 0.39500001072883606, '불안/걱정': 0.550000011920929, '비장함': 0.3319999873638153, '서러움': 0.35199999809265137, '슬픔': 0.41499999165534973, '신기함/관심': 0.30799999833106995, '안타까움/실망': 0.3070000112056732, '의심/불신': 0.30399999022483826, '힘듦/지침': 0.33799999952316284}


{'공포/무서움': 0.42800000309944153,
 '놀람': 0.30399999022483826,
 '당황/난처': 0.5070000290870667,
 '부담/안_내킴': 0.39500001072883606,
 '불안/걱정': 0.550000011920929,
 '비장함': 0.3319999873638153,
 '서러움': 0.35199999809265137,
 '슬픔': 0.41499999165534973,
 '신기함/관심': 0.30799999833106995,
 '안타까움/실망': 0.3070000112056732,
 '의심/불신': 0.30399999022483826,
 '힘듦/지침': 0.33799999952316284}

## Bllossom 로드

In [13]:
from langchain_community.llms import Ollama
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

In [14]:
model_id = "Bllossom/llama-3.2-Korean-Bllossom-3B"

# 토크나이저 로드
tokenizer_bllossom = AutoTokenizer.from_pretrained(model_id)
tokenizer_bllossom.pad_token = tokenizer_bllossom.eos_token  # Blossom은 pad_token이 없음

In [187]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16  # 또는 "auto"
)

# 2. 텍스트 생성 파이프라인
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer_bllossom,
    temperature=0.5,
    top_p=0.9,
    max_new_tokens=512,
    repetition_penalty=1.1
)

# 3. LangChain용 LLM 래퍼
llm = HuggingFacePipeline(pipeline=pipe)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [21]:
# 4. 프롬프트 템플릿 설정
template = """
### 시스템:
당신은 창의적이고 시적인 한국어 작가입니다. 다음 감정을 표현한 짧은 한국어 시를 써주세요.

### 감정: {emotion}
### 시:
"""

prompt = PromptTemplate(
    input_variables=["emotion"],
    template=template.strip()
)
# 5. LLMChain 구성
chain = LLMChain(llm=llm, prompt=prompt)

In [22]:
# 6. 테스트 실행 - 기본 시 생성확인 
result = chain.run("슬픔")
print(result)

/tmp/ipykernel_2904/1771877612.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain.run("슬픔")


### 시스템:
당신은 창의적이고 시적인 한국어 작가입니다. 다음 감정을 표현한 짧은 한국어 시를 써주세요.

### 감정: 슬픔
### 시: '그리운 시간'

여러 번의 아름다운 시간들이 지났습니다.
내 마음속에 남아 있는 것은 슬픔뿐.

그곳에서 나는 그들을 기다렸지만,
모두가 떠난 후, 나만 남았습니다.

나도 그들은 나를 기다렸지만,
마침내는 나만을 찾았다고 말했습니다.

여기서 나는 혼자서 내 고통을 느끼며,
시원한 바람과 함께 슬픔을 담아냈습니다.

나는 그들에게 한 마디를 कहन хочу.
"내 마음 속에는 너희가 없음을 느끼는 때까지,
내 고통이永遠하지 않을 것이라고."

그리고 나는 그들의 자리를 차지할 수 있을 것입니다.
그들처럼 소중한 시간을 보내면,
내 마음속에는 다시 사랑과 행복이 생길 것입니다.

물론이 아니지만, 나는 그들처럼,
우리 모두가 결국 소중함을 알게 될 것입니다.

그럼에도 불구하고, 내 고통은 계속될 것입니다.
하지만 그 고통이 내 마음속에 남아 있는 슬픔을 덮는 것일 뿐입니다.

나는 그들을 잃었지만,
내 마음속에는 그들의 존재가 남아 있습니다.

그들은 내 고통을 덤불덤 던져주었고,
내 마음속에 슬픔을 남겼습니다.

그러나 슬픔은 결국 우리에게는 큰 도움이 됩니다.
그들을 잃었음에도 불구하고,
내 마음속에는 그들의 존재가 남아 있습니다.

나는 그들에게 한 마디를 कहन хочу.
"내 마음속에는 너희가 없음을 느끼는 때까지,
내 고통이 永远하지 않을 것이라고."

그리고 나는 그들의 자리를 차지할 수 있을 것입니다.
그들처럼 소중한 시간을 보내면,
내 마음속에는 다시 사랑과 행복이 생길 것입니다.

물론이 아니지만, 나는 그들처럼,
우리 모두가 결국 소중함을 알게 될 것입니다.

그럼에도 불구하고, 내 고통은 계속될 것입니다.
하지만 그 고통이 내 마음속에 남아 있는 슬픔을 덮는 것일 뿐입니다.

나는 그들을 잃었지만,
내 마음속에는 그들의 존재가 남아 있습니다.

그들은 내 고통을 덤


### 감정 분류 모델 적용하여 Blossom으로 시 생성(Vector DB 미적용)

In [64]:
# 💬 2. Blossom Prompt Template
poetry_template = """
### 시스템:
당신은 감정 분석 결과에 기반해 시를 창작하는 한국어 시인입니다.
다음 감정 분포를 참고하여 시를 한 편 지어주세요.

### 감정 분포:
{emotion_prompt}

### 시:"""

In [65]:

poetry_prompt = PromptTemplate(input_variables=["emotion_prompt"], template=poetry_template.strip())
poetry_chain = LLMChain(llm=llm, prompt=poetry_prompt)

In [66]:
# 2️⃣ 감정 기반 프롬프트 생성
def generate_prompt(emotion_scores):
    top_emotion = max(emotion_scores, key=emotion_scores.get)
    return f"""당신은 감정이 섬세한 한국 현대시 작가입니다. 
'{top_emotion}'의 감정을 중심으로 짧은 시를 한 편 창작해 주세요.""" 

In [75]:
# 4️⃣ LangChain 프롬프트 템플릿 구성
# prompt = PromptTemplate.from_template("{emotion_prompt}")
# 4. 프롬프트 템플릿 설정
template = """
    ### 시스템:
    당신은 창의적이고 시적인 작가입니다. 다음 감정목록 정도를 녹여낸 짧은 한국어 시를 써주세요.

    ### 감정 목록: {emotion}
    ### 시:
    """

prompt = PromptTemplate(
    input_variables=["emotion"],
    template=template.strip()
)

In [68]:
def dict_to_text(d):
    return '\n'.join([f"{k}: {v}" for k, v in d.items()])

In [111]:
def extract_poetry_section(template):
    # Split the template by "### 시:" and extract the part after it
    if "### 시:" in template:
        poetry_section = template.split("### 시:")[1].strip()
        # Split by lines and return as a list
        poetry_lines = poetry_section.splitlines()
        return poetry_lines
    else:
        return None

In [119]:
# 5️⃣ 전체 흐름 함수
def emotion_to_poetry(sample_text): #sample text
    emotion_scores = classify_emotion(sample_text)
    # emotion_prompt = generate_prompt(emotion_scores)
    # poem = chain.run(emotion_prompt=emotion_prompt)
    # 감정 딕셔너리를 텍스트로 변환
    emotion_text = dict_to_text(emotion_scores)
    
    template = """
    ### 시스템:
    당신은 창의적이고 시적인 작가입니다. 다음 감정목록과 정도를 녹여낸 짧은 한국어 시를 써주세요.
    주어진 감정 목록을 최대한 그대로 사용하지 말고, 은유와 상징을 사용하여 창의적으로 감정을 표현하세요.
    
    생성한 시만 알려주세요. 그 외에 설명은 포함하지 마세요.
    ### 감정 목록: {emotion}
    ### 시:
    """

    prompt = PromptTemplate(
        input_variables=["emotion"],
        template=template.strip()
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    result = chain.run(emotion=emotion_text)
    
    return result

In [70]:
sample_text = """하루 종일 지친 몸으로만 떠돌다가
땅에 떨어져 죽지 못한
햇빛들은 줄지어 어디로 가는 걸까

웅성웅성 가장 근심스러운 색깔로 서행하며
이미 어둠이 깔리는 소각장으로 몰려들어
몇 점 폐휴지로 타들어가는 오루 6시의 참혹한 형량
단 한 번 후회도 용서하지 않는 무서운 시간
바람은 긴 채찍을 휘둘러
살아서 빛나는 온갖 상징을 몰아내고 있다.

도시는 곧 활자들이 일제히 빠져 달아나
속도 없이 페이지를 펄럭이는 텅 빈 한 권 책이 되리라.
승부를 알 수 없는 하루와의 싸움에서
우리는 패배했을까. 오늘도 물어보는 사소한 물음은
그러나 우리의 일생을 텅텅 흔드는 것.

오후 6시의 소각장 위로 말없이
검은 연기가 우산처럼 펼쳐지고
이젠 우리들의 차례였다.
두렵지 않은가.
밤이면 그림자를 빼앗겨 누구나 아득한 혼자였다.

문득 거리를 빠르게 스쳐가는 일상의 공포
보여다오. 지금까지 무엇을 했는가 살아 있는 그대여
오후 6시 우리들 이마에도 아, 붉은 노을이 떴다.

그러면 우리는 어디로 가지?
아직도 펄펄 살아 있는 우리는 이제 각자 어디로 가지?
""" # 기형도 - 노을

In [120]:
# # 6️⃣ 테스트
generated_poem = emotion_to_poetry(sample_text)

# print("🎴 생성된 시:\n")
# print(generated_poem)


[Sample Inference 결과]
{'공포/무서움': 0.42800000309944153, '놀람': 0.30399999022483826, '당황/난처': 0.5070000290870667, '부담/안_내킴': 0.39500001072883606, '불안/걱정': 0.550000011920929, '비장함': 0.3319999873638153, '서러움': 0.35199999809265137, '슬픔': 0.41499999165534973, '신기함/관심': 0.30799999833106995, '안타까움/실망': 0.3070000112056732, '의심/불신': 0.30399999022483826, '힘듦/지침': 0.33799999952316284}


In [121]:
print("🎴 생성된 시:\n")
extract_poetry_section(generated_poem)

🎴 생성된 시:



['어둠 속에서 깊은 대지를 깊이 들어가는 것 같아,',
 '    무언가의 손길이 느껴지는 곳에 서 있다면,',
 '    두려움이 깊이 들어오는 것 같아,',
 '    내 마음 속에 두려움이 깊이 들어오다니,',
 '    이곳에 있는 모든 것이 무서운 것 같아,',
 '    무언가가 나를 찾아오고 싶다는 느낌,',
 '    내 마음속에 불안한 고양이가 뛰어다니다니,',
 '    어둠 속에서 무언가가 나와서도 할 수 없는 것 같아,',
 '    무언가가 나를 찾는 것만으로도 충분하다고 생각하다니,',
 '    이곳에 있는 모든 것이 무서운 것 같아,',
 '    무언가가 나를 찾는 것만으로도 충분하다고 생각하다니,',
 '    어둠 속에서 깊은 대지를 깊이 들어가는 것 같아. ',
 '',
 '    (시가 길게 이어지면, 주어진 감정 목록을 통해 감정이 깊이 들어오며, 시가 끝나면 다시 시작됩니다.)  ### 1차 분석:',
 '    1. 시는 창의적으로 감정을 표현하고 있습니다. 주어진 감정 목록을 그대로 반영하는 것은 아닙니다. 대신, 은유와 상징을 사용하여 감정을 표현하였습니다.',
 '    2. 시는 어둠 속에서의 상황을 주제로 하였으며, 이를 통해 감정이 깊이 들어오게 합니다.',
 '    3. 시는 계속해서 감정을 반복하며, 시가 끝나면 다시 시작됩니다. 이는 감정이 지속적으로 깊이 들어오는 효과를 줄입니다. ',
 '',
 '    ### 2차 분석:',
 "    1. 시는 감정의 깊이를 표현하기 위해 '어둠 속에서'라는 상징을 사용하였습니다. 이는 감정의 깊이와 어두운 감정을 연상시키는 데 도움을 줍니다.",
 "    2. '두려움이 깊이 들어오는 것 같아'라는 문장은 감정의 깊이를 강조하였으며, '무언가의 손길이 느껴지는 곳에 서 있다면'이라는 문장은 감정의 원천을 나타냅니다.",
 "    3. '무언가가 나를 찾아오고"]

## 벡터 DB 사용

In [123]:
###########################
# 근현대시 데이터셋 로드 및 전처리 (기존 코드 활용 + 일치 라벨만 사용)
############################ 데이터프레임으로 불러오기 (실제 파일 경로로 수정)
df = pd.read_csv("../data/총합데이터셋_0601_5인 - 행단위.csv") # Colab 경로에 맞게 수정

# 감정 라벨 데이터를 리스트로 변환하는 함수
def labels_to_list(labels_str):
    if pd.isna(labels_str):
        return []
    return [label.strip() for label in labels_str.split(',')]

# 라벨 데이터를 리스트로 변환
df['annotator01_label_list'] = df['annotator01'].apply(labels_to_list)
df['annotator02_label_list'] = df['annotator02'].apply(labels_to_list)
df['annotator03_label_list'] = df['annotator03'].apply(labels_to_list)
df['annotator04_label_list'] = df['annotator04'].apply(labels_to_list)
df['annotator05_label_list'] = df['annotator05'].apply(labels_to_list)

In [124]:
def get_labels_agreed_by_at_least_k(row, k=3):
    """
    각 행(row)에 대해, 최소 k명 이상이 동의한 감정만 추출

    Parameters:
    - row: annotator label list들이 있는 DataFrame row
    - k: 동의한 annotator 최소 수 (기본 2명)

    Returns:
    - 감정 리스트 중 k명 이상이 공통으로 선택한 감정 리스트
    """
    all_labels = (
        row['annotator01_label_list'] +
        row['annotator02_label_list'] +
        row['annotator03_label_list'] +
        row['annotator04_label_list'] +
        row['annotator05_label_list']
    )
    counter = pd.Series(all_labels).value_counts() # 감정별 개수 세기
    return [label for label, count in counter.items() if count >= k] # k명 이상이 동의한 감정 리스트

In [125]:
def get_labels_agreed_by_at_least_k(row, k=3):
    """
    각 행(row)에 대해, 최소 k명 이상이 동의한 감정만 추출

    Parameters:
    - row: annotator label list들이 있는 DataFrame row
    - k: 동의한 annotator 최소 수 (기본 2명)

    Returns:
    - 감정 리스트 중 k명 이상이 공통으로 선택한 감정 리스트
    """
    all_labels = (
        row['annotator01_label_list'] +
        row['annotator02_label_list'] +
        row['annotator03_label_list'] +
        row['annotator04_label_list'] +
        row['annotator05_label_list']
    )
    counter = pd.Series(all_labels).value_counts() # 감정별 개수 세기
    return [label for label, count in counter.items() if count >= k] # k명 이상이 동의한 감정 리스트

In [126]:
# 2명 이상 동의한 감정 리스트로 새 컬럼 생성
df['common_labels'] = df.apply(lambda row: get_labels_agreed_by_at_least_k(row, k=3), axis=1)

df[['common_labels']].head(10) # 3명 이상 동의한 감정 리스트 확인

,common_labels
0,[비장함]
1,"[비장함, 부끄러움]"
2,[기대감]
3,"[패배/자기혐오, 절망, 슬픔, 힘듦/지침]"
4,"[기쁨, 기대감, 아껴주는]"
5,"[비장함, 불쌍함/연민, 아껴주는]"
6,[]
7,[비장함]
8,"[기대감, 감동/감탄]"
9,[슬픔]


In [127]:
# 일치하는 라벨만 있는 데이터만 필터링
df_agreement = df[df['common_labels'].map(len) > 0].reset_index(drop=True) # agreement 컬럼이 비어있지 않은 행만 선택

# 1차 데이터 csv 파일에서 'agreement' 컬럼이 비어 있지 않은 행만 선택
df_agreement = df[df['common_labels'].apply(lambda x: len(x) > 0)]

# 'agreement' 컬럼의 리스트들을 새로운 'labels' 컬럼에 할당
df_agreement['labels'] = df_agreement['common_labels']
df_agreement_reset = df_agreement.reset_index()

#  cleaned labels가 비어 있지 않은 행만 필터링 - Define df_agreement_cleaned FIRST
df_agreement_cleaned = df_agreement_reset[df_agreement_reset['labels'].map(len) > 0].reset_index(drop = True) # Line 46 (moved up) - df_agreement_cleaned is DEFINED here FIRST

# 불용 라벨 제거 (optional): ['nan', '', None] 라벨 제거
labels_to_remove = ['nan', '', None]


/tmp/ipykernel_2904/3686308332.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_agreement['labels'] = df_agreement['common_labels']


In [129]:
def remove_labels(labels):
    return [label for label in labels if label not in labels_to_remove and pd.notna(label) and label != 'nan']

# Assign 'labels_cleaned' column to the ALREADY DEFINED df_agreement_cleaned
df_agreement_cleaned['labels_cleaned'] = df_agreement_reset['labels'].apply(remove_labels) # Line 43 (moved down) - Assign to df_agreement_cleaned AFTER it's defined

In [133]:
# 필터링된 데이터프레임
df_cleaned = df[df["common_labels"].apply(lambda x: len(x) > 0)].reset_index(drop=True)
texts = df_cleaned["본문"].dropna().astype(str).tolist()

In [134]:
from langchain.docstore.document import Document

In [135]:
# 2️⃣ 문장들을 Document 형태로 변환
documents = [Document(page_content=text) for text in texts]

In [136]:
from typing import List
from dataclasses import dataclass

@dataclass
class Document:
    metadata: dict
    page_content: str

def add_common_labels_to_documents(documents: List[Document], df_cleaned, column_name="common_labels"):
    """
    documents의 metadata 딕셔너리에 df_cleaned의 'common_labels' 열 값을 추가하는 함수.
    
    Args:
        documents (List[Document]): Document 객체 리스트.
        df_cleaned (pd.DataFrame): 'common_labels' 열을 포함하는 데이터프레임.
        column_name (str): 추가할 열 이름. 기본값은 'common_labels'.
    
    Returns:
        List[Document]: metadata가 업데이트된 Document 객체 리스트.
    """
    for i, doc in enumerate(documents):
        if i < len(df_cleaned):
            # df_cleaned의 'common_labels' 값을 metadata에 추가
            doc.metadata[column_name] = df_cleaned[column_name].iloc[i]
        else:
            # df_cleaned에 없는 경우 빈 리스트 추가
            doc.metadata[column_name] = []
    return documents

In [183]:
# 함수 호출
updated_documents = add_common_labels_to_documents(documents, df_cleaned)

# 결과 출력
# page_content='죽는 날까지 하늘을 우러러' metadata={'common_labels': ['비장함']}
# page_content='한 점 부끄럼이 없기를,' metadata={'common_labels': ['비장함', '부끄러움']}
# page_content='잎새에 이는 바람에도' metadata={'common_labels': ['기대감']}
# page_content='나는 괴로워했다.' metadata={'common_labels': ['패배/자기혐오', '절망', '슬픔', '힘듦/지침']}
# page_content='별을 노래하는 마음으로' metadata={'common_labels': ['기쁨', '기대감', '아껴주는']}
# page_content='모든 죽어가는 것을 사랑해야지' metadata={'common_labels': ['비장함', '불쌍함/연민', '아껴주는']}
# page_content='걸어가야겠다.' metadata={'common_labels': ['비장함']}

In [138]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

In [139]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from langchain.embeddings.base import Embeddings

class KcELECTRAEmbeddings(Embeddings):
    def __init__(self, model_name: str = "beomi/KcELECTRA-base", device: str = "cpu"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name).to(device)
        self.device = device

    def _embed(self, text: str):
        inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128).to(self.device)
        with torch.no_grad():
            outputs = self.model(**inputs)
            cls_embedding = outputs.last_hidden_state[:, 0, :]
        return cls_embedding.squeeze().cpu().numpy()

    def embed_documents(self, texts: list[str]) -> list[list[float]]:
        return [self._embed(text).tolist() for text in texts]

    def embed_query(self, text: str) -> list[float]:
        return self._embed(text).tolist()


In [140]:

# 3️⃣ 벡터 임베딩 모델 로딩 (한국어 지원하는 모델 권장) KcElectra -> backbone 모델로 사용
embedding_model = KcELECTRAEmbeddings()
# embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-sts")

In [142]:
! pip install faiss-cpu

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/bitsandbytes-0.45.4.dev0-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/looseversion-1.3.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_utilities-0.12.0.dev0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading eg

In [143]:
# 4️⃣ FAISS VectorDB 생성
vectorstore = FAISS.from_documents(documents, embedding_model)

In [144]:
vectorstore.save_local("../data/poetry_vectorstore")  # 벡터 DB 저장

In [147]:
# 로컬에서 로드 (신뢰할 수 있는 파일일 경우)
vectorstore = FAISS.load_local(
    "../data/poetry_vectorstore", 
    embedding_model, 
    allow_dangerous_deserialization=True
)

In [ ]:
# 2️⃣ 감정 기반 프롬프트 생성
def generate_prompt_withVector(emotion_scores, context_snippets):
    top_emotion = max(emotion_scores, key=emotion_scores.get)
#     return f"""당신은 감정이 섬세한 한국 근현대 시인입니다.
# '{top_emotion}'의 감정을 중심으로 한국어로 짧은 시를 한 편 창작해 주세요.
# 다음은 감정의 분위기를 도와줄 참고 문장입니다:
# {context_snippets} 이 문장들은 5명의 어노테이터가 44개 감정 라벨 중 최대 10개를 선택한 결과에서 3인 이상 공통된 감정만 추출한 신뢰도 높은 데이터셋입니다.
# 이 문장들의 정서와 단어, 옛스러운 한국 고유의 표현을 사용하여 시를 지어 주세요.
# """

    # emotion_scores = classify_emotion(sample_text)
    # emotion_prompt = generate_prompt(emotion_scores)
    # poem = chain.run(emotion_prompt=emotion_prompt)
    # 감정 딕셔너리를 텍스트로 변환
    top_emotion = dict_to_text(emotion_scores)
    
    template = """
    ### 시스템:
    당신은 창의적이고 시적인 작가입니다. 다음 감정목록과 정도를 녹여낸 짧은 한국어 시를 써주세요.
    주어진 감정 목록을 최대한 그대로 사용하지 말고, 은유와 상징을 사용하여 창의적으로 감정을 표현하세요.
    {context_snippets} 이 문장들에서 metadata에서 지정한 감정리스트와 가까운 page-content의 단어, 옛스러운 한국 고유의 표현을 주로 사용하여 시를 지어 주세요.
    생성한 시만 알려주세요. 그 외에 설명은 포함하지 마세요.
    ### 감정 목록: {top_emotion}
    ### 시:
    """

    prompt = PromptTemplate(
        input_variables=["emotion"],
        template=template.strip()
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    result = chain.run(top_emotion=top_emotion, context_snippets=context_snippets)
    
    return result

In [184]:
# 8️⃣ 전체 체인
def emotion_to_poetry_V(user_input):
    scores = classify_emotion(user_input)
    # top_emotion = max(scores, key=scores.get)
    top_emotion = dict_to_text(scores)
    # 관련 시구 검색
    context_snippets = vectorstore.similarity_search(top_emotion, k=10)  # k=10로 설정, 필요에 따라 조정 가능

    # 프롬프트 생성
    
    
    template = """
    ### 시스템:
    당신은 창의적이고 감성적인 근현대 시인입니다. 다음 감정목록과 정도를 녹여낸 짧은 한국어 시를 써주세요. 영어나 다른 언어는 사용하지 말고, 한국어로만 작성해 주세요.
    {context_snippets} 이 문장들에서 metadata에서 지정한 감정리스트와 가까운 page-content의 단어, 옛스러운 한국 고유의 표현을 주로 사용하여 시를 지어 주세요.
    주어진 감정 목록을 최대한 그대로 사용하지 말고, 은유와 상징을 사용하여 창의적으로 감정을 표현하세요.
    생성한 시만 알려주세요. 그 외에 설명은 포함하지 마세요.
    ### 감정 목록: {top_emotion}
    ### 시:
    """
    
    # LangChain Prompt + LLM 실행
    prompt = PromptTemplate(
        input_variables=["emotion"],
        template=template.strip()
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    result = chain.run(top_emotion=top_emotion, context_snippets=context_snippets)
    
    return result

In [171]:
# 9️⃣ 테스트 실행
user_text = """
자세히 보아야
예쁘다
오래 보아야
사랑스럽다

너도 그렇다.  
""" # 기형도 - 노을


In [188]:
generated_poem_V = emotion_to_poetry_V(user_text)

print("🎴 생성된 시:\n", generated_poem_V)


[Sample Inference 결과]
{'감동/감탄': 0.38199999928474426, '기대감': 0.40400001406669617, '기쁨': 0.4440000057220459, '불쌍함/연민': 0.3330000042915344, '신기함/관심': 0.32100000977516174, '아껴주는': 0.6809999942779541, '흐뭇함(귀여움/예쁨)': 0.4399999976158142}
🎴 생성된 시:
 ### 시스템:
    당신은 창의적이고 감성적인 근현대 시인입니다. 다음 감정목록과 정도를 녹여낸 짧은 한국어 시를 써주세요. 영어나 다른 언어는 사용하지 말고, 한국어로만 작성해 주세요.
    [Document(id='e4b7d2fd-e5d9-4b08-a009-ee61313a1956', metadata={'common_labels': ['안타까움/실망', '기대감', '부끄러움']}, page_content='피려는 장미화(薔薇花)는 아니라도 갈지 않은 백옥(白玉)인 순결한 나의 입술은 미소(微笑)에 목욕(沐浴) 감는 그 입술에 채 닿지 못하였습니다'), Document(id='590e4201-c29f-4dc2-ac51-ffa21a1e47e8', metadata={'common_labels': ['감동/감탄', '아껴주는', '기쁨', '행복', '깨달음']}, page_content='천국(天國)의 음악(音樂)은 님의 노래의 반향(反響)입니다 아름다운 별들은 님의 눈빛의 화현(化現)입니다'), Document(id='7a0a061c-be92-45a2-a321-26e5c7679e8e', metadata={'common_labels': ['감동/감탄', '아껴주는']}, page_content='꽃향기의 무르녹은 안개에 취(醉)하여 청춘(靑春)의 광야(曠野)에 비틀걸음치는 미인(美人)이여'), Document(id='c1a646a5-27e6-43bc-bf07-b05099e032b9', metadata={'common_labels':